<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #5: TF-IDF
`Fecha de entrega: Septiembre 19, 2020. (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

In [2]:
import re
import numpy as np
import pandas as pd
import re
from unicodedata import normalize

from nltk.corpus import stopwords
stopwords_sp = stopwords.words('spanish')

from nltk.stem.snowball import SnowballStemmer
spanishStemmer=SnowballStemmer("spanish")

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics.pairwise import euclidean_distances

# Punto 1: Pre-Procesamiento

- `[18 pts]` Leer el archivo `Princesas.csv` usando `pandas` y crear una nueva columna con el texto en minúscula, sin caracteres especiales ni números, sin palabras vacias y hacer stemming de las palabras

In [3]:
data = pd.read_csv('archivos/Princesas.csv')
data

,Princesa,Personalidad
0,Blancanieves,Blancanieves es una princesa de noble cuna que...
1,Cenicienta,Cenicienta es inicialmente una sirvienta en su...
2,Aurora,"La Princesa Aurora, la Bella Durmiente, es la ..."
3,Bella,Bella es una muchacha que vive en la campiña f...
4,Jasmín,"Cuando se introdujo por primera vez, la Prince..."
5,Pocahontas,"El nombre de Pocahontas significa ""Pequeña Sil..."
6,Mulan,Mulan es atípica a los anteriores papeles feme...
7,Tiana,Es una joven camarera que sueña con ser dueña ...
8,Mérida,Mérida llama la atención por su característico...
9,Moana,"Moana, una joven de 16 años de edad, hija únic..."


In [13]:
def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1",normalize( "NFD", texto), 0, re.I)
    texto = re.sub(r"[\W\d_]+", " ",texto)
    texto = re.sub(r'[0-9]+', '', texto)
    texto = texto.split() 
    texto = [palabra for palabra in texto if palabra not in stopwords_sp] 
    texto = [spanishStemmer.stem(palabra) for palabra in texto] 
    texto = " ".join(texto)
    return texto

In [14]:
data['pre-procesado'] = data['Personalidad'].apply(lambda texto: pre_procesado(texto))

In [15]:
data

,Princesa,Personalidad,pre-procesado
0,Blancanieves,Blancanieves es una princesa de noble cuna que...,blancaniev princes nobl cun ve forz servidumbr...
1,Cenicienta,Cenicienta es inicialmente una sirvienta en su...,cenicient inicial sirvient cas constant objet ...
2,Aurora,"La Princesa Aurora, la Bella Durmiente, es la ...",princes auror bell durmient hij unic rein flor...
3,Bella,Bella es una muchacha que vive en la campiña f...,bell muchach viv campin frances padr inventor ...
4,Jasmín,"Cuando se introdujo por primera vez, la Prince...",introduj primer vez princes jasmin poc dias de...
5,Pocahontas,"El nombre de Pocahontas significa ""Pequeña Sil...",nombr pocahont signif pequ silenci bas figur h...
6,Mulan,Mulan es atípica a los anteriores papeles feme...,mul atip anterior papel femenin pelicul disney...
7,Tiana,Es una joven camarera que sueña con ser dueña ...,jov camarer suen ser duen propi restaur algun ...
8,Mérida,Mérida llama la atención por su característico...,mer llam atencion caracterist pel anaranj oscu...
9,Moana,"Moana, una joven de 16 años de edad, hija únic...",moan jov an edad hij unic sucesor import jef e...


# Punto 2: TF-IDF

- `[16 pts]` Crear la matriz TF-IDF

In [18]:
tfidf_vect = TfidfVectorizer()
tfidf = tfidf_vect.fit_transform(data['pre-procesado'].values)
tfidf_matrix = pd.DataFrame(tfidf.toarray(), index=data['Princesa'].values, columns=tfidf_vect.get_feature_names())
tfidf_matrix.T.round(3)

,Blancanieves,Cenicienta,Aurora,Bella,Jasmín,Pocahontas,Mulan,Tiana,Mérida,Moana
abrum,0.000,0.0,0.065,0.000,0.000,0.000,0.0,0.000,0.00,0.000
abuel,0.000,0.0,0.000,0.000,0.000,0.000,0.0,0.000,0.00,0.101
acuerd,0.000,0.0,0.055,0.000,0.000,0.000,0.0,0.000,0.08,0.000
adem,0.000,0.0,0.000,0.000,0.000,0.097,0.0,0.000,0.00,0.000
ador,0.064,0.0,0.048,0.063,0.000,0.000,0.0,0.000,0.00,0.000
...,...,...,...,...,...,...,...,...,...,...
viv,0.000,0.0,0.000,0.072,0.000,0.000,0.0,0.000,0.00,0.086
volunt,0.000,0.0,0.000,0.000,0.163,0.000,0.0,0.000,0.00,0.000
volv,0.000,0.0,0.065,0.000,0.000,0.000,0.0,0.000,0.00,0.000
vudu,0.000,0.0,0.000,0.000,0.000,0.000,0.0,0.126,0.00,0.000


# Punto 3: Distancia del coseno
- `[12 pts]` Calcular la distancia del coseno entre cada una de las princesas
- `[2 pts]` ¿Cuáles son las princesas más parecidas?
- `[2 pts]` ¿Cuáles son las princesas más diferentes?

In [21]:
dist_cos = cosine_distances(tfidf_matrix.values)
dist_cos = pd.DataFrame(dist_cos, columns = data['Princesa'].values, index = data['Princesa'].values)
dist_cos

,Blancanieves,Cenicienta,Aurora,Bella,Jasmín,Pocahontas,Mulan,Tiana,Mérida,Moana
Blancanieves,0.000000,0.849005,0.840957,0.919716,0.935414,0.921354,0.873673,0.961721,0.985396,0.958921
Cenicienta,0.849005,0.000000,0.855388,0.943448,0.958420,0.956574,0.933945,0.918273,0.970544,0.974982
Aurora,0.840957,0.855388,0.000000,0.803022,0.936960,0.865422,0.910704,0.918934,0.938838,0.961955
Bella,0.919716,0.943448,0.803022,0.000000,0.889229,0.865845,0.879901,0.963351,0.947661,0.908024
Jasmín,0.935414,0.958420,0.936960,0.889229,0.000000,0.943590,0.936092,0.935021,0.972123,0.969104
Pocahontas,0.921354,0.956574,0.865422,0.865845,0.943590,0.000000,0.877391,0.962937,0.963890,0.917667
Mulan,0.873673,0.933945,0.910704,0.879901,0.936092,0.877391,0.000000,0.980113,0.987560,0.947832
Tiana,0.961721,0.918273,0.918934,0.963351,0.935021,0.962937,0.980113,0.000000,0.978574,0.970930
Mérida,0.985396,0.970544,0.938838,0.947661,0.972123,0.963890,0.987560,0.978574,0.000000,0.965289
Moana,0.958921,0.974982,0.961955,0.908024,0.969104,0.917667,0.947832,0.970930,0.965289,0.000000


In [30]:
np.min(dist_cos[dist_cos>0]) 

Blancanieves    0.840957
Cenicienta      0.849005
Aurora          0.803022
Bella           0.803022
Jasmín          0.889229
Pocahontas      0.865422
Mulan           0.873673
Tiana           0.918273
Mérida          0.938838
Moana           0.908024
dtype: float64

Las princesas más parecidas son Aurora y bella

In [31]:
np.max(dist_cos)

Blancanieves    0.985396
Cenicienta      0.974982
Aurora          0.961955
Bella           0.963351
Jasmín          0.972123
Pocahontas      0.963890
Mulan           0.987560
Tiana           0.980113
Mérida          0.987560
Moana           0.974982
dtype: float64

Las princesas más diferentes son Merida y Mulan 